In [ ]:
# read the graph
import graph_tool.all as gt
from tqdm_pickle import load_file
import moduleG


rev_map = load_file("revmap.pkl")
moduleG.reverse_map = rev_map
print("The dict has been loaded!")
graph = gt.load_graph("BitcoinGraph.gt")
print("The graph has been loaded!")
from moduleG import *

In [ ]:
# processing structure indicators (serial computing)

'''
    need to set the number of nodes of k-hop subgraph
'''

import traceback
import importlib
importlib.reload(moduleG)
moduleG.reverse_map = rev_map
from moduleG import *
from tqdm.notebook import tqdm
import pandas as pd


def nhop(g, address, n):
    source = moduleG.reverse_map["account_dict"][address]
    dist = {source:0}
    mp = {source:0}
    num_nhop = 0
    def mp_add(x, num):
        if x not in mp:
            mp[x] = len(mp)
            num += 1
        return num
    edges = []
    gv = gt.GraphView(g, directed=False)
    for f,t in gt.bfs_iterator(gv,source):
        if f not in dist:
            break
        if dist[f] < n:
            if t not in dist:
                dist[t] = dist[f] + 1
                num_nhop = mp_add(t, num_nhop)
            if g.edge(f, t):
                edges.append((mp[f], mp[t]))
            if g.edge(t, f):
                edges.append((mp[t], mp[f]))
        if num_nhop > 3000: # set the number k of k-hop subgraph
            break
    ng = gt.Graph()
    ng.add_edge_list(edges)
    ng.vp["address"] = ng.new_vp("string")
    for i in mp.items():
        ng.vp["address"][i[1]] = g.vp["address"][i[0]]
    return ng


# </editor-fold>
from tqdm.notebook import tqdm
tqdm.pandas()
def account_Statistics(path_folder:str=None, result_folder:str=None):
    start_time = time.time()
    data = pd.read_csv(path_folder, index_col="account")
    def f1(df):
        try:
            s_graph = nhop(graph, df.name, 4)
            df['S1-1'], df['S1-2'], df['S1-3'], df['S1-4'], df['S1-5'], df['S1-6'] = module_221(s_graph)
            df['S2-1'], df['S2-2'], df['S2-3'] = module_222(s_graph)
            df['S3'] = module_223(s_graph)
            df['S4'] = module_224(s_graph)
            df['S5'] = module_225(s_graph)
            df['S6'] = module_226(s_graph)
            # df['S7'] = module_231(s_graph)
            df['S8'] = module_232(s_graph)
            df['S9'] = module_233(s_graph)
        except BaseException as e:
            print(e, df.name)
            exstr = traceback.format_exc()
            print(exstr)
        return df
    data = data.progress_apply(f1, axis=1)
    data.to_csv(result_folder, mode='w')
    end_time = time.time()
    print("over:", end_time-start_time, "s")
    
account_Statistics("file_path", "final_file_path")

In [ ]:
# processing structure indicators (parallel computing)
import traceback
import importlib
importlib.reload(moduleG)
moduleG.reverse_map = rev_map
from moduleG import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=5)
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()


gv = gt.GraphView(graph,directed=False)
def nhop(g, address, n):
    source = moduleG.reverse_map["account_dict"][address]
    dist = {source:0}
    mp = {source:0}
    num_nhop = 0
    def mp_add(x, num):
        if x not in mp:
            mp[x] = len(mp)
            num += 1
        return num
    edges=[]
    for f,t in gt.bfs_iterator(gv, source):
        if f not in dist:
            break
        if dist[f] < n:
            if t not in dist:
                dist[t] = dist[f] + 1
                num_nhop = mp_add(t, num_nhop)
            if g.edge(f, t):
                edges.append((mp[f], mp[t]))
            if g.edge(t, f):
                edges.append((mp[t], mp[f]))
        if num_nhop > 3000: # set the number k of k-hop subgraph
            break
    ng = gt.Graph()
    ng.add_edge_list(edges)
    ng.vp["address"] = ng.new_vp("string")
    for i in mp.items():
        ng.vp["address"][i[1]] = g.vp["address"][i[0]]
    return ng


def mapline(df):
    index = df.name
    try:
        s_graph = nhop(graph, df.name, 4)
        df['S1-1'], df['S1-2'], df['S1-3'], df['S1-4'], df['S1-5'], df['S1-6'] = module_221(s_graph)
        df['S2-1'], df['S2-2'], df['S2-3'] = module_222(s_graph)
        df['S3'] = module_223(s_graph)
        df['S4'] = module_224(s_graph)
        df['S5'] = module_225(s_graph)
        df['S6'] = module_226(s_graph)
        # df['S7'] = module_231(s_graph)
        df['S8'] = module_232(s_graph)
        df['S9'] = module_233(s_graph)
    except BaseException as e:
        print(e, index)
        exstr = traceback.format_exc()
        print(exstr)
        print("___________")
        pass
    return df


def account_Statistics_multithreading(path_folder:str=None, result_folder:str=None):
    start_time = time.time()
    data = pd.read_csv(path_folder, index_col="account")
    data = data.parallel_apply(mapline, axis=1)
    data.to_csv(result_folder, mode='w')
    end_time = time.time()
    print("over:", end_time-start_time, "s")

account_Statistics_multithreading("file_path", "final_file_path")

In [ ]:
# fixing moudule to fill the extended indicators (parallel computing)
import traceback
import importlib
importlib.reload(moduleG)
moduleG.reverse_map = rev_map
from moduleG import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False, nb_workers=5)
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()


def nhop(g,address,n):
    source = moduleG.reverse_map["account_dict"][address]
    dist = {source:0}
    mp = {source:0}
    num_nhop = 0
    def mp_add(x, num):
        if x not in mp:
            mp[x] = len(mp)
            num += 1
        return num
    edges = []
    gv = gt.GraphView(g,directed=False)
    for f,t in gt.bfs_iterator(gv, source):
        if f not in dist:
            break
        if dist[f] < n:
            if t not in dist:
                dist[t] = dist[f] + 1
                num_nhop = mp_add(t, num_nhop)
            if g.edge(f,t):
                edges.append((mp[f], mp[t]))
            if g.edge(t,f):
                edges.append((mp[t], mp[f]))
        if num_nhop > 3000: # set the number k of k-hop subgraph
            break
    ng = gt.Graph()
    ng.add_edge_list(edges)
    ng.vp["address"] = ng.new_vp("string")
    for i in mp.items():
        ng.vp["address"][i[1]] = g.vp["address"][i[0]]
    return ng
def mapline(df):
    index = df.name
    try:
        # df['CI2a3-1'], df['CI2a3-2'] = module_14213(graph, index)
        df['CI2a31-1'], df['CI2a31-2'] = module_142131(graph, index)
        df['CI2a32-1'], df['CI2a32-2'], df['CI2a32-3'], df['CI2a32-4'] = module_142132(graph, index)
        df['CI2a33-1'], df['CI2a33-2'] = module_142133(graph, index)
        # df['CI4a3'] = module_14413(graph, index)
        df['CI4a31'] = module_144131(graph, index)
        df['CI4a32-1'], df['CI4a32-2'] = module_144132(graph, index)
        df['CI4a33'] = module_144133(graph, index)
        # df['CI4a4'] = module_14414(graph, index)
        df['CI4a41'] = module_144141(graph, index)
        df['CI4a42-1'], df['CI4a42-2'] = module_144142(graph, index)
        df['CI4a43'] = module_144143(graph, index)
        
        s_graph = nhop(graph,df.name,4)
        # df['S1-1'], df['S1-2'], df['S1-3'], df['S1-4'], df['S1-5'], df['S1-6'] = module_221(s_graph)
        # df['S2-1'], df['S2-2'], df['S2-3'] = module_222(s_graph)
        # df['S3'] = module_223(s_graph)
        # df['S4'] = module_224(s_graph)
        df['S5'] = module_225(s_graph)
        # df['S6'] = module_226(s_graph)
        # df['S7'] = module_231(s_graph)
        # df['S8'] = module_232(s_graph)
        # df['S9'] = module_233(s_graph)
        df['S10'] = module_234(s_graph)
    except BaseException as e:
        print(e, index)
        exstr = traceback.format_exc()
        print(exstr)
        print("___________")
        pass
    return df


def account_Statistics_multithreading(path_folder:str=None, result_folder:str=None):
    start_time = time.time()
    data = pd.read_csv(path_folder,index_col="account")
    data['S10'] = None
    data = data.parallel_apply(mapline,axis=1)
    data.to_csv(result_folder, mode='w')
    end_time = time.time()
    print("over:", end_time-start_time,"s")

account_Statistics_multithreading("file_path","final_file_path")

In [ ]:
# processing statistical indicators (parallel computing)
imp.reload(moduleG)
moduleG.reverse_map = rev_map
from moduleG import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
tqdm.pandas()


def mapline(df):
    index = df.name
    try:
        df['PAIa11-1'], df['PAIa11-2'] = module_11311(graph, index)
        df['PAIa12'] = module_11312(graph, index)
        df['PAIa13'] = module_11313(graph, index)
        df['PAIa14-1'], df['PAIa14-2'], df['PAIa14-3'], df['PAIa14-4'] = module_11314(graph, index)
        df['PAIa14-R1'], df['PAIa14-R2'], df['PAIa14-R3'], df['PAIa14-R4'] = module_11314(graph, index, flag_repeat=False)
        df['PAIa15-1'], df['PAIa15-2']= module_11315(graph, index)
        df['PAIa15-R1'], df['PAIa15-R2'] = module_11315(graph, index, flag_repeat=False)
        df['PAIa16-1'], df['PAIa16-2'] = module_11316(graph, index)
        df['PAIa16-R1'], df['PAIa16-R2'] = module_11316(graph, index, flag_repeat=False)
        df['PAIa17-1'], df['PAIa17-2'], df['PAIa17-3'] = module_11317(graph, index)
        df['PAIa17-R1'], df['PAIa17-R2'], df['PAIa17-R3'] = module_11317(graph, index, flag_repeat=False)
        df['PAIa21-1'], df['PAIa21-2'], df['PAIa21-3'], df['PAIa21-4'] = module_11321(graph, index)
        df['PAIa21-R1'], df['PAIa21-R2'], df['PAIa21-R3'], df['PAIa21-R4'] = module_11321(graph, index, flag_repeat=False)
        df['PAIa22-1'], df['PAIa22-2'] = module_11322(graph, index)
        df['PAIa22-R1'], df['PAIa22-R2'] = module_11322(graph, index, flag_repeat=False)
        df['PDIa1-1'], df['PDIa1-2'], df['PDIa1-3'] = module_1231(graph, index)
        df['PDIa1-R1'], df['PDIa1-R2'], df['PDIa1-R3'] = module_1231(graph, index, drop_duplicated=True)
        df['PDIa11-1'], df['PDIa11-2'] = module_12311(graph,index)
        df['PDIa11-R1'], df['PDIa11-R2'] = module_12311(graph,index, drop_duplicated=True)
        df['PDIa12'] = module_12312(graph, index)
        df['PDIa12-R'] = module_12312(graph, index, drop_duplicated=True)
        df['PDIa13'] = module_12313(graph, index)
        df['PDIa13-R'] = module_12313(graph, index, drop_duplicated=True)
        df['PTIa1'] = module_1311(graph, index)
        df['PTIa2'] = module_1312(graph, index)
        df['PTIa21'] = module_13121(graph, index)
        # df['PTIa3'] = module_1313(graph,index)
        df['PTIa31-1'], df['PTIa31-2'], df['PTIa31-3'] = module_13131(graph, index)
        df['PTIa32'] = module_13132(graph, index)
        df['PTIa33'] = module_13133(graph, index)
        # df['PTIa4'] = module_1314(graph, index)
        df['PTIa41-1'], df['PTIa41-2'], df['PTIa41-3'] = module_13141(graph, index)
        df['PTIa42'] = module_13142(graph, index)
        df['PTIa43'] = module_13143(graph, index)
        df['CI1a1-1'], df['CI1a1-2'] = module_14131(graph, index)
        # df['CI1a1-1'], df['CI1a1-2'] = module_14131(graph, index)
        df['CI1a2'] = module_14132(graph, index)
        # df['CI2a1'] = module_14211(graph, index)
        df['CI2a11-1'], df['CI2a11-2'] = module_142111(graph, index)
        df['CI2a12-1'], df['CI2a12-2'], df['CI2a12-3'], df['CI2a12-4'] = module_142112(graph, index)
        # df['CI2a2'] = module_14212(graph, index)
        df['CI2a21-1'], df['CI2a21-2'] = module_142121(graph, index)
        df['CI2a22-1'], df['CI2a22-2'], df['CI2a22-3'], df['CI2a22-4'] = module_142122(graph, index)
        df['CI2a23-1'], df['CI2a23-2'] = module_142123(graph, index)
        # df['CI2a3-1'], df['CI2a3-2'] =module_14213(graph, index)
        df['CI2a31-1'], df['CI2a31-2'] = module_142131(graph, index)
        df['CI2a32-1'], df['CI2a32-2'], df['CI2a32-3'], df['CI2a32-4'] = module_142132(graph, index)
        df['CI2a33-1'], df['CI2a33-2'] = module_142133(graph, index)
        # df['CI3a1'] = module_14311(graph, index)
        df['CI3a11-1'], df['CI3a11-2'] = module_143111(graph,index)
        df['CI3a12-1'], df['CI3a12-2'], df['CI3a12-3'], df['CI3a12-4'] = module_143112(graph, index)
        # df['CI3a2'] = module_14312(graph,index)
        df['CI3a21-1'], df['CI3a21-2'], df['CI3a21-3'] = module_143121(graph,index)
        df['CI3a22-1'], df['CI3a22-2'], df['CI3a22-3'], df['CI3a22-4'], df['CI3a22-5'], df['CI3a22-6'] = module_143122(graph, index)
        df['CI3a23-1'], df['CI3a23-2'], df['CI3a23-3'] = module_143123(graph,index)
        # df['CI3a3-1'], df['CI3a3-2'] = module_14313(graph, index)
        df['CI3a31-1'], df['CI3a31-2'] = module_143131(graph, index)
        df['CI3a32-1'], df['CI3a32-2'], df['CI3a32-3'], df['CI3a32-4'] = module_143132(graph, index)
        df['CI3a33-1'], df['CI3a33-2'] = module_143133(graph,index)    
        # df['CI4a1'] = module_14411(graph, index)
        df['CI4a11'] = module_144111(graph, index)
        df['CI4a12-1'], df['CI4a12-2'] = module_144112(graph, index)
        df['CI4a13'] = module_144113(graph, index)
        # df['CI4a2'] = module_14412(graph, index)
        df['CI4a21'] = module_144121(graph, index)
        df['CI4a22-1'], df['CI4a22-2'] = module_144122(graph, index)
        df['CI4a23'] = module_144123(graph, index)
        # df['CI4a3'] = module_14413(graph, index)
        df['CI4a31'] = module_144131(graph, index)
        df['CI4a32-1'], df['CI4a32-2'] = module_144132(graph, index)
        df['CI4a33'] = module_144133(graph, index)
        # df['CI4a4'] = module_14414(graph, index)
        df['CI4a41'] = module_144141(graph, index)
        df['CI4a42-1'], df['CI4a42-2'] = module_144142(graph, index)
        df['CI4a43'] = module_144143(graph, index)
    except BaseException as e:
        print(e, index)
        exstr = traceback.format_exc()
        print(exstr)
        print("___________")
        pass
    return df


def account_Statistics_multithreading(path_folder:str=None, result_folder:str=None):
    start_time = time.time()
    data = pd.read_csv(path_folder, index_col="account")
    data = data.parallel_apply(mapline, axis=1)
    data.to_csv(result_folder, mode='w')
    end_time = time.time()
    print("over:", end_time-start_time, "s")

account_Statistics_multithreading("file_path", "final_file_path")